This is a follow-along notebook of [Tools in LlamaIndex](https://colab.research.google.com/#fileId=https%3A//huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/tools.ipynb) from [Hugging Face Agents Course](https://huggingface.co/learn/agents-course/unit2/llama-index/tools), with additional trials. 

# Tools in LlamaIndex
There are four main types of tools in LlamaIndex:
1.  `FunctionTool`: Convert any Python function into a tool that an agent can use. It automatically figures out how the function works.
2.  `QueryEngineTool`: A tool that lets agents use query engines. Since agents are built on query engines, they can also use other agents as tools.
3.  `Toolspecs`: Sets of tools created by the community, which often include tools for specific services like Gmail.
4.  `Utility Tools`: Special tools that help handle large amounts of data from other tools.

## Install Dependencies

In [1]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface llama-index-tools-google -Uqq

## Create a Function Tool

In [2]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str: 
    """Useful for getting weather information for a given location."""
    print(f"Getting weather for {location}...")
    return f"The weather in {location} is cloudy."

tool = FunctionTool.from_defaults(
    get_weather, 
    name="my_weather_tool", 
    description="Useful for getting weather information for a given location."
)

tool.call("Hong Kong")

Getting weather for Hong Kong...


ToolOutput(content='The weather in Hong Kong is cloudy.', tool_name='my_weather_tool', raw_input={'args': ('Hong Kong',), 'kwargs': {}}, raw_output='The weather in Hong Kong is cloudy.', is_error=False)

In [3]:
def bmi_calculator(weight: float, height: float):
    """
    This tool calculates the Body Mass Index (BMI) based on weight (kilograms) and height (meters).
    It returns a BMI category (Underweight, Normal, Overweight, or Obese).
    """
    bmi = weight / (height ** 2)

    if bmi <= 18.5:
        return "Underweight"
    elif bmi <= 25.0:
        return "Normal"
    elif bmi <= 30.0:
        return "Overweight"
    else:
        return "Obese"
    
tool = FunctionTool.from_defaults(
    bmi_calculator, 
    name="bmi", 
    description="""This tool calculates the Body Mass Index (BMI) based on weight (kilograms) and height (meters).
    It returns a BMI category (Underweight, Normal, Overweight, or Obese).
    """
)

tool.call(70, 1.6)

ToolOutput(content='Overweight', tool_name='bmi', raw_input={'args': (70, 1.6), 'kwargs': {}}, raw_output='Overweight', is_error=False)

## Create a QueryEngineTool

In [ ]:
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = Ollama(model="qwen3:8b", request_timeout=60.0)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="persona_db",
    description="This tool query persona databse.",
)
await tool.acall(
    "Responds about research on the impact of AI on the future of work and society?"
)

ToolOutput(content="<think>\nOkay, the user is asking about how to respond to research on AI's impact on the future of work and society, but they provided context about a social justice educator focused on diversity, equity, and inclusion. Let me check the context again.\n\nThe context mentions someone working with families and communities to promote empathy and understanding of intersectional identity and oppression. So the answer should tie AI's impact to these areas. Maybe discuss how AI could affect marginalized groups, access to opportunities, or equity in the workforce. Need to ensure it's from the perspective of someone advocating for DEI. Avoid mentioning the context directly. Focus on how AI might exacerbate existing inequalities or create new challenges in equity and inclusion. Also, consider the societal implications like bias in AI systems and the need for inclusive policies. Make sure the answer aligns with the educator's role in addressing these issues through education a

In [19]:
tool.metadata

ToolMetadata(description='some useful description', name='some useful name', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [6]:
result = await tool.acall(
    "With the persona of a data analyst, what tool do they frequently use?"
)

In [7]:
result

ToolOutput(content="<think>\nOkay, let's see. The user is asking about a data analyst's frequently used tool, but the context provided is about a web developer or student focused on HTML, CSS, and website building. The persona mentioned doesn't relate to data analysis. Since the instructions say to use only the given context and not prior knowledge, I can't assume any tools related to data analysis. The answer should stick to the context, which doesn't mention data analysts. So, there's no information here about data analysts or their tools. I need to make sure not to reference the context directly and avoid mentioning the context in the answer. Maybe the answer is that the context doesn't provide info on data analysts, but the user wants a tool. Wait, but the persona is a web developer, so maybe the tools they use are related to web development like code editors or browsers. But the question specifically mentions a data analyst. Since the context doesn't cover that, I should state tha

In [8]:
print(result)

<think>
Okay, let's see. The user is asking about a data analyst's frequently used tool, but the context provided is about a web developer or student focused on HTML, CSS, and website building. The persona mentioned doesn't relate to data analysis. Since the instructions say to use only the given context and not prior knowledge, I can't assume any tools related to data analysis. The answer should stick to the context, which doesn't mention data analysts. So, there's no information here about data analysts or their tools. I need to make sure not to reference the context directly and avoid mentioning the context in the answer. Maybe the answer is that the context doesn't provide info on data analysts, but the user wants a tool. Wait, but the persona is a web developer, so maybe the tools they use are related to web development like code editors or browsers. But the question specifically mentions a data analyst. Since the context doesn't cover that, I should state that the provided inform

In [9]:
result.content

"<think>\nOkay, let's see. The user is asking about a data analyst's frequently used tool, but the context provided is about a web developer or student focused on HTML, CSS, and website building. The persona mentioned doesn't relate to data analysis. Since the instructions say to use only the given context and not prior knowledge, I can't assume any tools related to data analysis. The answer should stick to the context, which doesn't mention data analysts. So, there's no information here about data analysts or their tools. I need to make sure not to reference the context directly and avoid mentioning the context in the answer. Maybe the answer is that the context doesn't provide info on data analysts, but the user wants a tool. Wait, but the persona is a web developer, so maybe the tools they use are related to web development like code editors or browsers. But the question specifically mentions a data analyst. Since the context doesn't cover that, I should state that the provided info

In [10]:
result.is_error

False

## Create Toolspecs

In [13]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
tool_spec_list

To get a more detailed view of the tools, we can take a look at the `metadata` of each tool.

In [15]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query[str]: The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n            subject (Optiona

In [17]:
tool_spec_list[0].metadata

ToolMetadata(description="load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account.", name='load_data', fn_schema=<class 'llama_index.core.tools.utils.load_data'>, return_direct=False)

## Model Context Protocol (MCP) in LlamaIndex

In [23]:
!pip install llama-index-tools-mcp -Uqq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# Get agent
agent = await get_agent(mcp_tool)

# Create agent context
agnet_context = Context(agent)

NameError: name 'get_agent' is not defined

## Utility Tools
You can find toolspecs and utility tools on the [LlamaHub](https://llamahub.ai/). 